# Ejemplo: Vectorización de textos
**Autor:** Unidad de Científicos de Datos (UCD)

---
Este ejemplo muestra las principales funcionalidades del módulo `limpieza`, de la librería **ConTexto**. También se muestran ejemplos de uso de las funciones de limpieza contenidas en el módulo auxiliar `limpieza_aux`, que hace parte de `utils`.

Para mayor información sobre estos módulos y sus funciones, se puede consultar [su documentación](link de la página de docs).

In [ ]:
import os
import sys

# Añadir la ubiación del paquete al path actual, para poder hacer las
# importaciones
scripts_path = os.path.abspath(os.path.join('../Contexto'))
if scripts_path not in sys.path:
    sys.path.insert(0, scripts_path)

from contexto.limpieza import limpieza_texto, lista_stopwords
from contexto.vectorizacion import *

# Corpus de prueba
textos_prueba = [
    'Este es el primer texto de prueba para la vectorización y sus elementos.',
    'Una segunda oración permite evaluar si hay elementos en común para vectorizar.',
    'Tercera frase que consiste en un texto complementario con palabras comúnmente utilizadas.',
    'En esta oración y la siguiente se introducen elementos para completar un grupo de por lo menos 5.',
    'Finalmente, esta frase cierra un grupo de 5 oraciones para probar la vectorización.',
    'Una última frase para ampliar un poco el grupo.']

# Limpieza básica a los textos para quitar ruido
textos_limpios = [limpieza_texto(i, lista_stopwords(), quitar_numeros=False) for i in textos_prueba]

# Texto que no hace parte del corpus original
texto_nuevo = 'hola, esta es una frase de prueba para aplicar la vectorización'

### 1. Vectorizadores por frecuencia de términos ###

## 1.1 Inicializar los vectorizadores
v_bow = VectorizadorFrecuencias()
# Este tiene en cuenta palabras y bigramas, y solo coge las 20 más frecuentes
v_tfidf = VectorizadorFrecuencias('tfidf', rango_ngramas=(1, 2), max_elementos=20)

## 1.2 Ajustar los vectorizadores
# Se van a guardar los vectorizadores ajustados en archivos para su posterior uso
v_bow.ajustar(textos_limpios, archivo_salida='out/v_bow.pk')
v_tfidf.ajustar(textos_limpios, archivo_salida='out/v_tfidf.pk')

## 1.3 Vocabulario de un vectorizador entrenado
print(v_bow.vocabulario())
print(v_tfidf.vocabulario())

## 1.4 Vectorizar textos utilizando los vectorizadores entrenados
vector_bow = v_bow.vectorizar(texto_nuevo, disperso=True)  # Salida como matriz dispersa
vector_tfidf = v_tfidf.vectorizar(texto_nuevo, disperso=False)  # Salida como un numpy array

## 1.5 Transformada inversa
'''
Nótese que al realizar la transformada inversa se pierde el orden de las palabras.
Esto se debe a que estos métodos de vectorización no tienen en cuenta el orden
sino la frecuencia de aparición de cada término.
Además, si un término no está en el vocabulario del vectorizador, no va a estar
incluído en el vector y por lo tanto no se podrá recuperar en la transformada
inversa.
'''
print(textos_limpios[0])
print(v_bow.inversa(v_bow.vectorizar(textos_prueba))[0])

print(textos_limpios[2])
print(v_tfidf.inversa(v_tfidf.vectorizar(textos_prueba))[2])

## 1.6 Cargar un vectorizador ajustado previamente
'''
Al cargar un vectorizador ajustado previamente (mediante el parámetro "archivo_modelo")
Los demás parámetros de inicialización no serán tenidos en cuenta, pues esos parámetros
se tomarán del vectorizador cargado.
'''
v_bow_2 = VectorizadorFrecuencias(archivo_modelo='out/v_bow.pk')
v_tfidf_2 = VectorizadorFrecuencias(archivo_modelo='out/v_tfidf.pk')

# Se vectoriza el mismo texto con los vectorizadores cargados
vector_bow_2 = v_bow_2.vectorizar(texto_nuevo, disperso=True)  # Salida como matriz dispersa
vector_tfidf_2 = v_tfidf_2.vectorizar(texto_nuevo, disperso=False)  # Salida como un numpy array

# Se comprueba que los vectores resultantes sean iguales
np.all((vector_bow == vector_bow_2).toarray())
np.all(vector_tfidf == vector_tfidf_2)

### 2. Vectorizador por hashing ###

## 2.1 Inicializar el vectorizador
# Se define que los vectores tendrán 50 elementos
v_hash = VectorizadorHash(n_elementos=50)

## 2.2 Vectorizar textos utilizando el vectorizador
'''
A pesar de que el objeto HashingVectorizer de la librería scikit-learn
incluye la función "fit", esta en la práctica no hace nada, pues este
tipo de vectorizador funciona sin vocabulario y no debe ser ajustado.
Por lo tanto, en la librería no se ha incluído esta opción. En vez de eso,
el objeto se aplica directamente a los textos para vectorizarlos.

Por este mismo motivo, el VectorizadorHash no permite la transformación
inversa para identificar las palabras de un vector.
'''
vectores_prueba = v_hash.vectorizar(textos_prueba)
vectores_prueba.shape

vector_nuevo = v_hash.vectorizar(texto_nuevo, disperso=False)
vector_nuevo.shape

### 3. Vectorizador por Word2Vec ###

## 3.1 Inicializar el vectorizador
v_word2vec = VectorizadorWord2Vec()

## 3.2 Vectorizar textos utilizando el vectorizador
'''
Este vectorizador obtiene los vectores de cada palabra de un texto, y luego las 
promedia para obtener un único vector para el texto completo.
'''
vector = v_word2vec.vectorizar(texto_nuevo)
vector.shape

## 3.3 Textos con palabras desconocidas (no incluídas en el modelo)
'''
El argumento booleano 'quitar_desconocidas' en la función vectorizar_texto hará que
la función sea ligeramente más demorada, pero quizás más precisa, al no tener en cuenta palabras
que no están incluídas en el modelo. Cuando este argumento es False (valor por defecto),
para cada palabra desconocida se incluirá un vector de solo ceros, lo que afectará el vector 
promedio resultante.
'''

texto_1 = 'En este texto todas las palabras son conocidas, por lo que los resultados deberían ser iguales'
texto_2 = 'En este texto hay asfafgf términos desconocidos FGs<g gsi<gi<sbf'

for i, t in enumerate([texto_1, texto_2]):
    print('\n------------------')
    print(f'Texto {i+1}:')
    print(f'"{t}"')
    v1 = v_word2vec.vectorizar(t, quitar_desconocidas=False)
    v2 = v_word2vec.vectorizar(t, quitar_desconocidas=True)
    print(f'Diferencia promedio: {(v1 - v2).mean()}')

## 3.4 Obtener palabras y vectores de un texto
df_palabras = v_word2vec.vectores_palabras(texto_nuevo, tipo='dataframe')
dict_palabras = v_word2vec.vectores_palabras(texto_nuevo, tipo='diccionario')

## 3.5 Similitudes entre textos
'''
Esta función aprovecha las facilidades de la librería Spacy para medir la
similaridad entre 2 palabras o textos.
'''
t1 = 'los perros y los gatos suelen pelear mucho.'
t2 = 'caninos y felinos entran en disputas con frecuencia.'
t3 = 'este tercer texto habla sobre un tema distinto a los otros dos'

for i in [t1, t2]:
    for j in [t2, t3]:
        if i != j:
            similitud = v_word2vec.similitud_textos(i, j)
            print('-----------------------')
            print(f'Texto 1: {i}')
            print(f'Texto 2: {j}')
            print(f'Similitud entre textos: {similitud}')

### 4. Vectorizador por Doc2Vec ###

## 4.1 Inicializar el vectorizador
# Se configura para que tenga 100 elementos y se entrene por 25 épocas
'''
Dado que el "corpus" de entrenamiento va a se pequeño (5 textos cortos), puede haber 
error si ningúna palabra cumple con el parámetro "minima_cuenta=5" (valor por defecto).
Para evitar este error en este caso, se cambia ese parámetro a 1 (valor mínimo).
'''
v_doc2vec = VectorizadorDoc2Vec(n_elementos=100, epocas=25, minima_cuenta=1)

## 4.2 Entrenar el modelo en un corpus
v_doc2vec.entrenar_modelo(textos_limpios, archivo_salida='out/v_doc2vec.pk')

## 4.3 Vectorizar textos utilizando el vectorizador
vector = v_doc2vec.vectorizar(texto_nuevo)
vector.shape

## 4.4 Cargar un vectorizador entrenado previamente
v_doc2vec_2 = VectorizadorDoc2Vec(archivo_modelo='out/v_doc2vec.pk')

# Se vectoriza el mismo texto con el vectorizador cargado
vector_2 = v_doc2vec_2.vectorizar(texto_nuevo)

# Se comprueba que ambos vectores resultantes sean iguales
np.all(vector == vector_2)

____
## 1. Funciones de limpieza de textos

En esta sección se muestra cómo se pueden hacer distintos procesamientos de un texto de entrada para remover elementos como signos de puntuación, *stopwords*, números y acentos, que pueden llegar a entorpecer el análisis de un conjunto de documentos.

### 1.1 Importar módulo de ConTexto y definir texto de prueba

In [1]:
from contexto.limpieza import *

texto_prueba = '''hola, esto es una prueba para verificar que la limpieza
sea hecha con precisión, empeño y calidad! Esperamos que esté todo de 10.

Desde Amazonas hasta la Guajira y san andrés, desde John y María hasta Ernesto,
esperamos       que todo funcione de manera correcta.'''

ModuleNotFoundError: No module named 'contexto'

### 1.2 Aplicar funciones de limpieza de textos

In [ ]:
# Limpieza básica, se pasa todo a minúsculas y se eliminan signos de puntuación
limpio_basico = limpieza_basica(texto_prueba)

# Si se desea mantener los caracteres numéricos
limpio_basico_nums = limpieza_basica(texto_prueba, quitar_numeros=False)

# Para quitar acentos (diéresis, tildes y virgulillas)
sin_acentos = remover_acentos(limpio_basico)

# Quitar palabras con menos de 4 caracteres
quitar_0a3_caracteres = remover_palabras_cortas(sin_acentos, 4)


Utilizando la función `limpieza_texto` se puede, a la vez:

* Pasar todo el texto a minúsculas
* Quitar signos de puntuación
* Quitar *stopwords* (palabras y/o expresiones). Para esto, se pueden pasar directamente las listas de palabras y expresiones a quitar, o se puede pasar un archivo que contenga esta información.
* Quitar palabras de una longitud menor a *n* caracteres (configurable)
* Quitar números (configurable)
* Quitar acentos (configurable)


In [ ]:
limpio_completo = limpieza_texto(texto_prueba, ubicacion_archivo='in/stopwords_prueba.txt', n_min=3)
print(limpio_completo)

### 1.3 Quitar elementos repetidos de un texto

La función `quitar_repetidos` permite quitar elementos repetidos de un texto, de acuerdo a un separador definido por el usuario.

In [ ]:
texto_repetido = 'hola, hola, como estas,hola, hola tu'

# Aplicar función directamente
quitar_repetidos(texto_repetido)

# Especificar el separador entre documentos/frases
quitar_repetidos(texto_repetido, ',')

# Deshabilitar opción de quitar espacios al inicio y al final
quitar_repetidos(texto_repetido, ',', remover_espacios=False) 

### 1.4 Cargar listas de *stopwords*, predefinidas y definidas por el usuario

**ConTexto** trae algunas listas predefinidas de *stopwords* que pueden ser cargadas y utilizadas directamente. Las listas incluidas son:

* Palabras comunes del lenguaje castellano (solo palabras)
* Nombres comunes de hombres y mujeres (solo palabras)
* Nombres de municipios y departamentos de Colombia (palabras y expresiones: nombres compuestos como "San Andrés")

Además de estas listas, la función `lista_stopwords` permite cargar listas predefinidas de las *stopwords* más comunes para varios lenguajes, utilizando la librería NLTK.

Finalmente, la función `cargar_stopwords` permite al usuario cargar *stopwords* (tanto palabras como expresiones) desde un archivo plano. Las palabras/expresiones deben ir separadas por comas o ir en renglones separados para ser tenidas en cuenta por aparte.


In [ ]:
# Cargar listas de stopwords predefinidas
nombres_hombres = lista_nombres('hombre')
nombres_mujeres = lista_nombres('mujer')
nombres_todos = lista_nombres()
apellidos = lista_apellidos()
municipios = lista_geo_colombia('municipios')
departamentos = lista_geo_colombia('departamentos')
todos_geo = lista_geo_colombia()

# Stopwords comunes de varios lenguajes (por defecto se devuelven las de español)
stopwords = lista_stopwords()
stopwords_ingles = lista_stopwords('ingles')

# Cargar archivo con lista de términos y expresiones que se desean remover
custom_sw = cargar_stopwords('entrada/stopwords_prueba.txt')
print(custom_sw)

____
## 2. Funciones auxiliares para limpieza de textos

Adicionalmente, el módulo auxiliar `limpieza_aux` contiene algunas funciones complementarias que permiten identificar y remover elementos adicionales que puedan entorpecer el análisis de un conjunto de textos.

### 2.1 Importar funciones auxiliares y definir textos de prueba

In [ ]:
from contexto.utils.limpieza_aux import substrings_en_comun, detectar_coincidencias
from contexto.utils.limpieza_aux import caracteres_repetidos, caracteres_consecutivos, consonantes_consecutivas
from contexto.utils.limpieza_aux import quitar_coincidenias, quitar_palabras_atipicas

# Corpus de prueba
textos_prueba = [
    'Este es el primer texto de prueba para la detección de coincidencias.',
    'Una segunda oración permite evaluar si hay cadanea de caracteres elementos en común.',
    'Tercera frase que consiste en un texto complementario con palabras comúnmente utilizadas.',
    'En esta oración y la siguiente se introducen elementos para completar un grupo de por lo menos 5.',
    'Finalmente, esta frase cierra un grupo de 5 oraciones para probar la detección de coincidencias.',
    'Una última frase para ampliar un poco el grupo.']


### 2.2 Detectar y quitar coincidencias entre un conjunto de textos

En ocasiones un documento puede tener un encabezado o pie de nota común en casi todas sus páginas. Esto puede entorpecer ciertos análisis, al darle un peso demasiado grande a estas coincidencias.

Para evitar este problema, la función `quitar_coincidenias` (que a su vez utiliza las funciones `substrings_en_comun` y `detectar_coincidencias`) permite, para un conjunto de textos, encontrar y remover coincidencias (cadenas de caracteres) que cumplan una o varias de estas condiciones:

* Que aparezcan en mínimo una proporción determinada de todos los textos
* Que su longitud (cantidad de caracteres) sea mayor o igual a un valor determinado
* Que la cadena tenga un número de palabras mayor o igual a un valor determinado

In [3]:
# Detectar coincidencias de por lo menos 4 y 10 caracteres
print(substrings_en_comun(textos_prueba[4], textos_prueba[5], longitud_min=4))
print(substrings_en_comun(textos_prueba[4], textos_prueba[5], longitud_min=10))

# Detectar cadenas de caracteres de mínimo 2 palabras que estén en mínimo la mitad de los textos
print(detectar_coincidencias(textos_prueba, prop=0.5, n_min=2, longitud_min=5))

# Quitar las coincidencias encontradas
print(quitar_coincidenias(textos_prueba, prop=0.5, n_min=2, longitud_min=5))


NameError: name 'substrings_en_comun' is not defined

### 2.3 Detectar y quitar palabras o valores atípicos

Si se está trabajando con un texto de mala calidad (por ejemplo, porque se aplicó OCR a un documento antiguo y mal escaneado), es posible que haya "ruido" en el texto, como palabras sin sentido, que puede afectar el análisis de este documento. Otro caso posible es trabajar con textos que tengan palabras o valores numéricos sospechosos (como "abcde" o "0000000"). En este caso, puede ser de utilidad poder detectar y/o remover estas palabras sospechosas o de ruido.

Para evitar este problema, la función `quitar_palabras_atipicas` (que a su vez utiliza las funciones `caracteres_repetidos`, `caracteres_consecutivos` y `consonantes_consecutivas`) permite, para un conjunto de textos, encontrar y remover palabras que cumplan una o varias de estas condiciones:

* Que tengan un número o letra repetidos de forma seguida más veces de lo permitido
* Que tengan números o letras consecutivas de forma seguida en un número mayor de lo permitido
* Que tengan más consonantes seguidas de lo permitido


In [ ]:
# Detectar si una palabra tiene una cantidad determinada de caracteres repetidos seguidos
caracteres_repetidos('123444321', 4)
# La función por defecto quita acentos y pasa todo a minúsculas, para que esto no afecte la búsqueda de repetidos
caracteres_repetidos('GóOol', 3)

# Detectar si una palabra tiene una cantidad determinada de caracteres consecutivos seguidos
caracteres_consecutivos('123444321', 4)
caracteres_consecutivos('aBCdE', 4)

# Detectar si una palabra tiene una cantidad determinada de consonantes seguidas
consonantes_consecutivas('AbStracto', 3)
consonantes_consecutivas('Lynyrd Skynyrd', 4)
# El resultado cambia si se deja de incluir la letra "Y" como vocal
consonantes_consecutivas('Lynyrd Skynyrd', 4, incluir_y=False)
# La función quita acentos por defecto, por lo que puede trabajar con consonantes que tengan algún tipo de acento o tilde
consonantes_consecutivas('mñçs', 4)

# Prueba de quitar palabras con problemas en un texto
texto_prueba = 'HolaAá! esta es una pruebba para ver si, En 12345, se pueden abstraer las reglas del abcdario.'

texto_sin_atipicas = quitar_palabras_atipicas(texto_prueba, n_repetidas=3, n_consecutivas=3, n_consonantes=4)

print(f"---------------\nTexto original:\n{texto_prueba}")
print(f"---------------\nTexto sin palabras detectadas como atípicas:\n{texto_sin_atipicas}")